# Hierarchical Ar(1) on a Grid Test with STAN

In this notebook, we will test the compositional score matching on a hierarchical problem defined on a grid.
- The observations are on grid with `n_grid` x `n_grid` points.
- The global parameters are the same for all grid points with hyper-priors:
$$ \alpha \sim \mathcal{N}(0, 1) \quad
  \mu_\beta \sim \mathcal{N}(0, 0.1) \quad
  \log\text{std}_\beta \sim \mathcal{N}(\log(0.1), 0.5);$$

- The local parameters are different for each grid point
$$ \beta_{i,j} \sim \mathcal{N}(\mu_\beta, \text{std}_\beta^2)$$

-  In each grid point, we have a time series of `T` observations. For the time beeing, we fix $\sigma=0.1$.
$$ y_{i,j} \sim \mathcal{N}(\alpha + \beta_{i,j}y_{i,j-1}, \sigma^2)$$
- We observe $T=10$ time points for each grid point. We can also amortize over the time dimension.

In [ ]:
import os

import numpy as np

os.environ['KERAS_BACKEND'] = 'torch'
from bayesflow import diagnostics

from problems.ar1_grid import Prior, get_stan_posterior
from problems import plot_shrinkage, visualize_simulation_output

In [ ]:
T = 25
N = 64
data_samples = 100

prior = Prior()

global_param_names = prior.global_param_names
local_param_names = prior.get_local_param_names(N)

In [ ]:
results = []
for _ in range(100):
    test = prior.sample(1, n_local_samples=N, n_time_points=T)
    results.append(test['data'][0])
    #visualize_simulation_output(test['data'][0])

#print(test['global_params'])
results = np.stack(results, axis=0)

visualize_simulation_output(np.max(results, axis=0))
visualize_simulation_output(np.min(results, axis=0))
visualize_simulation_output(np.mean(results, axis=0))
visualize_simulation_output(np.std(results, axis=0))

In [ ]:
np.random.seed(42)

global_posteriors = []
local_posteriors = []

true_global = []
true_local = []

for i in range(data_samples):
    print(i)
    sample = prior.sample(1, n_local_samples=N, n_time_points=T)
    true_global.append(sample['global_params'].flatten().cpu().numpy())
    true_local.append(sample['local_params'].flatten().cpu().numpy())
    sim_test = sample['data'][0].cpu().numpy()

    global_posterior, local_posterior = get_stan_posterior(sim_test, sigma_noise=prior.simulator.sigma_noise)
    global_posteriors.append(global_posterior)
    local_posteriors.append(local_posterior)

# make numpy arrays
global_posterior = np.stack(global_posteriors, axis=0)
local_posterior = np.stack(local_posteriors, axis=0).transpose(0, 2, 1)

true_global = np.stack(true_global, axis=0)
true_local = np.stack(true_local, axis=0)

# Plotting

In [ ]:
diagnostics.recovery(global_posterior, true_global, variable_names=global_param_names)
diagnostics.calibration_ecdf(global_posterior, true_global, difference=True, variable_names=global_param_names);

In [ ]:
diagnostics.recovery(local_posterior[:, :, :10], true_local[:, :10], variable_names=local_param_names[:10]);

In [ ]:
local_posterior_raw = prior.back_transform_local_params(local_posterior)

plot_shrinkage(global_posterior[:12, :, 1:], local_posterior_raw[:12, :, :, np.newaxis], min_max=(-5, 5))